In [151]:
import sys
import os

# Ruta al directorio raíz del proyecto (ajústala si es necesario)
ruta_raiz = os.path.abspath(r"C:\Users\Germán Llorente\Desktop\germiprogramer\TFG-Agente-de-Scouting")

if ruta_raiz not in sys.path:
    sys.path.append(ruta_raiz)

In [152]:
import importlib
import funciones_analisis.estadisticas

importlib.reload(funciones_analisis.estadisticas)
importlib.reload(funciones_analisis.generales)

from funciones_analisis.estadisticas import *
from funciones_analisis.generales import *
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch
from highlight_text import ax_text, fig_text
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.patheffects as path_effects
import seaborn as sns
import glob

In [153]:
# Cargar los datasets
jugadores_laliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_laliga.csv', encoding="utf-8", skipinitialspace=True)
jugadores_premier = pd.read_csv('datos/datos_jugadores_v3/jugadores_premier.csv')
jugadores_seriea = pd.read_csv('datos/datos_jugadores_v3/jugadores_seriea.csv')
jugadores_bundesliga = pd.read_csv('datos/datos_jugadores_v3/jugadores_bundesliga.csv')

In [154]:
for i, df in enumerate([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga], start=1):
    print(f"\nDataset {i} columnas:")
    print(list(df.columns))



Dataset 1 columnas:
['player_id', 'player_name', 'team', 'team_id', 'posiciones', 'partidos_jugados', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob', 'height_cm', 'weight_kg', 'club_jersey_number', 'club_loaned_from', 'club_contract_valid_until', 'nationality_id', 'nationality_name', 'preferred_foot', 'release_clause_eur', 'competition', 'tackle_success_rate', 'tackles_successful', 'interception_success_rate', 'interceptions', 'clearances', 'blocks', 'head_clearances', 'head_clearances_won', 'head_clearance_success_rate', 'headed_shots_total', 'headed_shots_after_duel', 'headed_shot_duel_rate', 'total_passes', 'completed_passes', 'incomplete_passes', 'passes_out', 'offside_passes', 'failed_passes', 'pass_completion_rate', 'avg_pass_length', 'ground_passes', 'low_passes', 'high_passes', 'ground_pass_percentage', 'low_pass_percentage', 'high_pass_percentage', 'crosses_total', 'crosses_completed', 'cutbacks_total', 'cutbacks_completed', 'switches_total', 'switches_complete

In [155]:
# Unir todos los datasets (uno debajo del otro)
jugadores_total = pd.concat([jugadores_laliga, jugadores_premier, jugadores_seriea, jugadores_bundesliga])
jugadores_total

,player_id,player_name,team,team_id,posiciones,partidos_jugados,overall,potential,value_eur,wage_eur,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,posicion_mas_jugada
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,['Left Back'],21,71,76,1800000.0,20000.0,...,0.598772,0.783009,0.000000,0.017631,-0.017503,0.000000,0.000000,0.460594,0.0,Left Back
1,3063,Danilo Luiz da Silva,Real Madrid,220,"['Right Back', 'Left Back', 'Right Wing']",24,80,86,16500000.0,90000.0,...,0.505618,0.716292,0.000000,0.040390,0.043820,0.000000,0.000000,0.252809,0.0,Right Back
2,3084,Christian Atsu Twasam,Málaga,223,"['Left Midfield', 'Right Midfield', 'Left Cent...",11,63,69,375000.0,4000.0,...,1.956522,1.956522,0.000000,0.410496,0.241304,0.000000,0.000000,0.000000,0.0,Right Midfield
3,3130,Gaël Kakuta,Sevilla,213,"['Left Midfield', 'Right Wing']",2,76,78,6500000.0,60000.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.967742,0.0,Left Midfield
4,3160,Rene Krhin,Granada,1049,"['Right Defensive Midfield', 'Left Defensive M...",23,75,80,4500000.0,35000.0,...,0.240642,0.240642,0.000000,0.031147,-0.031283,0.000000,0.000000,0.180481,0.0,Right Defensive Midfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,400621,Tohouri Zahoui Constant Djakpa,Eintracht Frankfurt,184,"['Left Back', 'Left Wing Back', 'Left Midfield...",11,75,75,3100000.0,40000.0,...,0.851351,0.851351,0.000000,0.019767,-0.019459,0.000000,0.000000,0.243243,0.0,Left Back
424,400622,Joel Gerezgiher,Eintracht Frankfurt,184,['Left Midfield'],3,60,72,300000.0,2000.0,...,0.978261,0.978261,0.000000,0.304519,-0.303261,0.000000,0.000000,0.978261,0.0,Left Midfield
425,400627,Dario Kresic,Bayer Leverkusen,904,['Goalkeeper'],1,68,68,675000.0,10000.0,...,0.000000,0.000000,1.914894,0.000000,0.000000,1.780851,-0.134043,0.000000,0.0,Goalkeeper
426,400654,Roel Brouwers,Borussia Mönchengladbach,185,"['Right Center Back', 'Left Center Back', 'Rig...",8,76,76,2300000.0,50000.0,...,0.000000,0.000000,0.000000,0.025924,-0.025806,0.000000,0.000000,0.000000,0.0,Right Center Back


In [156]:
import pandas as pd

# Lista de (player_id, team) válidos
ids_y_equipos_validos = [
    (3083, "Tottenham Hotspur"),
    (3089, "Manchester City"),
    (3302, "Watford"),
    (5474, "Inter Milan"),
    (5687, "Eibar"),
    (6593, "Real Betis"),
    (6743, "Rayo Vallecano"),
    (6950, "Swansea City"),
    (7141, "Levante UD"),
    (7170, "Carpi"),
    (7788, "Torino"),
    (7852, "Genoa"),
    (8217, "Juventus"),
    (9461, "Norwich City"),
    (10881, "Sevilla"),
    (17520, "VfB Stuttgart"),
    (18404, "Rayo Vallecano"),
    (23488, "Watford"),
    (26211, "Levante UD"),
    (27341, "Celta Vigo"),
    (27853, "Fiorentina"),
]

# Convertir a DataFrame
df_validos = pd.DataFrame(ids_y_equipos_validos, columns=['player_id', 'team'])

# Filtrar duplicados por player_id
duplicados = jugadores_total[jugadores_total.duplicated(subset='player_id', keep=False)]

# Mantener solo los duplicados correctos (player_id + team que coincidan con la lista)
duplicados_validos = duplicados.merge(df_validos, on=['player_id', 'team'])

# Mantener los jugadores únicos que no están duplicados
no_duplicados = jugadores_total[~jugadores_total['player_id'].isin(duplicados['player_id'])]

# Unir los válidos con los no duplicados
jugadores_total = pd.concat([no_duplicados, duplicados_validos], ignore_index=True)

# Ver resultado



In [157]:
# Filtrar jugadores que están cedidos (club_loaned_from no es NaN)
cedidos = jugadores_total[jugadores_total['club_loaned_from'].notna()]

# Mostrar algunas columnas relevantes
print(cedidos[['player_id', 'player_name', 'team', 'club_loaned_from']])


      player_id                    player_name                    team  \
10         3379                Alphonse Areola              Villarreal   
23         4418           Matheus Dória Macedo                 Granada   
65         5678   Jefferson Andrés Lerma Solís              Levante UD   
68         5719        Marco Asensio Willemsen                Espanyol   
70         5724                    Fayçal Fajr  RC Deportivo La Coruña   
...         ...                            ...                     ...   
1738      10902                Philipp Hosiner                 FC Köln   
1782      16502            László Kleinheisler           Werder Bremen   
1889       6743  Manuel Rolando Iturra Urrutia          Rayo Vallecano   
1890       7141                 Giuseppe Rossi              Levante UD   
1899       9461                     Timm Klose            Norwich City   

            club_loaned_from  
10       Paris Saint-Germain  
23    Olympique de Marseille  
65            Atlé

In [158]:
jugadores_total["team"].unique()

array(['Real Sociedad', 'Real Madrid', 'Málaga', 'Sevilla', 'Granada',
       'Valencia', 'Villarreal', 'Barcelona', 'Real Betis', 'Las Palmas',
       'Athletic Club', 'Rayo Vallecano', 'Espanyol', 'Atlético Madrid',
       'Celta Vigo', 'RC Deportivo La Coruña', 'Levante UD', 'Eibar',
       'Sporting Gijón', 'Getafe', 'Chelsea', 'West Bromwich Albion',
       'Manchester United', 'Tottenham Hotspur', 'Aston Villa',
       'Newcastle United', 'AFC Bournemouth', 'Manchester City',
       'West Ham United', 'Crystal Palace', 'Southampton', 'Sunderland',
       'Everton', 'Swansea City', 'Watford', 'Stoke City',
       'Leicester City', 'Norwich City', 'Liverpool', 'Arsenal', 'Torino',
       'Lazio', 'Fiorentina', 'AS Roma', 'AC Milan', 'Empoli', 'Genoa',
       'Inter Milan', 'Juventus', 'Atalanta', 'Udinese', 'Napoli',
       'Sassuolo', 'Hellas Verona', 'Palermo', 'Sampdoria', 'Bologna',
       'Chievo', 'Carpi', 'Frosinone', 'Ingolstadt', 'Hannover 96',
       'Schalke 04', 'Bayer 

In [159]:
jugadores_total["club_loaned_from"].unique()

array([nan, 'Paris Saint-Germain', 'Olympique de Marseille',
       'Atlético Huila', 'Real Madrid CF', 'Elche CF', 'AC Milan',
       'SC Braga', 'Atlético de Madrid', 'Rubin Kazan', 'Granada CF',
       'Derby County', 'Valencia CF', 'FC Porto', 'Watford', 'SL Benfica',
       'Roma', 'FC Barcelona', 'Inter', 'Liverpool', 'Galatasaray SK',
       'SV Werder Bremen', 'Villarreal CF', "CD O'Higgins", 'Sporting CP',
       'Celtic', 'Udinese Calcio', 'Cruz Azul', 'Norwich City',
       'Sunderland', 'Swansea City', 'AS Monaco', 'Tottenham Hotspur',
       'Chelsea', 'Stade Rennais FC', 'Fulham', 'Arsenal',
       'West Ham United', 'VfB Stuttgart', 'Zenit St. Petersburg',
       'Borussia Dortmund', 'U.S. Sassuolo Calcio', 'Palermo', 'Pescara',
       'Napoli', 'U.C. Sampdoria', 'Genoa', 'Juventus', 'Cagliari',
       'Grasshopper Club Zürich', 'FC Sion', 'Fenerbahçe SK',
       'Hellas Verona', 'Standard de Liège', 'Club Brugge KV',
       'Sevilla FC', 'Bologna', 'FC Bayern München',


In [160]:
import numpy as np

# Función para truncar a 2 decimales (sin redondear) y formatear como string con dos decimales
def truncar_y_formatear(x):
    return f"{np.floor(x * 100) / 100:.2f}"

# Detectar columnas numéricas con decimales (float)
columnas_decimales = jugadores_total.select_dtypes(include='float').columns

# Aplicar el truncado y formateo
for col in columnas_decimales:
    jugadores_total[col] = jugadores_total[col].apply(truncar_y_formatear)

# Ver resultado en las columnas modificadas
print(jugadores_total[columnas_decimales].head())


     value_eur  wage_eur club_jersey_number club_contract_valid_until  \
0   1800000.00  20000.00              19.00                   2016.00   
1  16500000.00  90000.00              23.00                   2021.00   
2    375000.00   4000.00              27.00                   2016.00   
3   6500000.00  60000.00              12.00                   2019.00   
4   4500000.00  35000.00              21.00                   2019.00   

  release_clause_eur tackle_success_rate interception_success_rate  \
0                nan               56.86                     62.75   
1                nan               64.18                     61.11   
2                nan               66.67                     66.67   
3                nan              100.00                      0.00   
4                nan               64.70                     71.05   

  head_clearance_success_rate headed_shot_duel_rate pass_completion_rate  ...  \
0                       51.43                  0.00        

In [161]:
#jugadores_total.to_csv("datos/datos_jugadores_v3/jugadores_total.csv")

In [162]:
df_ratings = pd.read_csv("datos/datos_jugadores_v3/jugadores_mundial.csv")

In [163]:
df_ratings

,Unnamed: 0,competition,date,match,team,pos,pos_role,player_name,match_id,minutes_played,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,posicion_mas_jugada
0,4886,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DC,Davinson Sánchez Mina,7585,120,...,0.000000,0.000000,0.0,0.138741,-0.135000,0.0,0.0,0.000000,0.0,Left Center Back
1,4890,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DC,Yerry Fernando Mina González,7585,120,...,0.000000,0.000000,0.0,0.127085,0.622500,0.0,0.0,0.000000,0.0,Right Center Back
2,4893,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DL,Johan Andrés Mojica Palacio,7585,120,...,1.500000,1.500000,0.0,0.004635,-0.007500,0.0,0.0,0.000000,0.0,Left Back
3,4897,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,DF,DR,Santiago Arias Naranjo,7585,116,...,0.775862,0.775862,0.0,0.000000,0.000000,0.0,0.0,0.775862,0.0,Right Back
4,4901,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,FW,FW,Juan Guillermo Cuadrado Bello,7585,120,...,2.250000,3.000000,0.0,0.076088,-0.075000,0.0,0.0,0.750000,0.0,Right Wing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1701,3024,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,MC,Hussain Al Mogahwi,7559,90,...,2.000000,2.000000,0.0,0.169422,-0.170000,0.0,0.0,0.000000,0.0,Right Center Midfield
1702,3026,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,MC,Salman Mohammed Al Faraj,7559,90,...,5.000000,5.000000,0.0,0.969338,0.030000,0.0,0.0,0.000000,0.0,Left Center Midfield
1703,3029,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,ML,Salem Mohammed Al Dawsari,7559,90,...,2.000000,2.000000,0.0,0.196278,0.800000,0.0,0.0,0.000000,0.0,Left Midfield
1704,3032,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Saudi Arabia,MF,MR,Hattan Babhir,7559,65,...,0.000000,0.000000,0.0,0.116950,-0.110769,0.0,0.0,0.000000,0.0,Right Midfield


In [164]:
jugadores_total.rename(columns={'posicion_mas_jugada': 'main_position'}, inplace=True)
df_ratings.rename(columns={'posicion_mas_jugada': 'main_position'}, inplace=True)

In [165]:
df_ratings[df_ratings["main_position"] == "Goalkeeper"]


,Unnamed: 0,competition,date,match,team,pos,pos_role,player_name,match_id,minutes_played,...,key_passes_per90,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,main_position
6,4909,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",Colombia,GK,GK,David Ospina Ramírez,7585,90,...,0.0,0.0,1.000000,0.0,0.0,0.793316,-0.210000,0.0,0.0,Goalkeeper
20,4967,World Cup 2018,2018-07-03,"Colombia - England, 1 - 1",England,GK,GK,Jordan Pickford,7585,90,...,0.0,0.0,1.000000,0.0,0.0,0.227255,-0.770000,0.0,0.0,Goalkeeper
34,3903,World Cup 2018,2018-06-28,"England - Belgium, 0 - 1",Belgium,GK,GK,Thibaut Courtois,7570,90,...,0.0,0.0,0.000000,0.0,0.0,0.460367,0.460000,0.0,0.0,Goalkeeper
48,3963,World Cup 2018,2018-06-28,"England - Belgium, 0 - 1",England,GK,GK,Jordan Pickford,7570,90,...,0.0,0.0,1.000000,0.0,0.0,0.152233,-0.850000,0.0,0.0,Goalkeeper
62,5049,World Cup 2018,2018-07-03,"Sweden - Switzerland, 1 - 0",Sweden,GK,GK,Robin Olsen,7586,90,...,0.0,0.0,0.000000,0.0,0.0,0.087946,0.090000,0.0,0.0,Goalkeeper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,3387,World Cup 2018,2018-06-26,"Denmark - France, 0 - 0",France,MF,DMC,Steve Mandanda,7563,90,...,0.0,0.0,0.000000,0.0,0.0,0.008513,0.010000,0.0,0.0,Goalkeeper
1658,2743,World Cup 2018,2018-06-24,"Japan - Senegal, 2 - 2",Japan,GK,GK,Eiji Kawashima,7556,90,...,0.0,0.0,2.000000,0.0,0.0,0.923394,-1.080000,0.0,0.0,Goalkeeper
1673,2790,World Cup 2018,2018-06-24,"Japan - Senegal, 2 - 2",Senegal,MF,MC,Khadim N''Diaye,7556,81,...,0.0,0.0,2.222222,0.0,0.0,0.909205,-1.311111,0.0,0.0,Goalkeeper
1685,2984,World Cup 2018,2018-06-25,"Saudi Arabia - Egypt, 2 - 1",Egypt,GK,GK,Essam Kamal Tawfik El Hadary,7559,90,...,0.0,0.0,2.000000,0.0,0.0,1.898497,-0.100000,0.0,0.0,Goalkeeper


In [166]:
jugadores_total["minutes_played"] = pd.to_numeric(jugadores_total["minutes_played"], errors="coerce")
jugadores_total["aerial_dominance_index_per90"] = pd.to_numeric(jugadores_total["aerial_dominance_index"], errors="coerce")

df_ratings["minutes_played"] = pd.to_numeric(df_ratings["minutes_played"], errors="coerce")
df_ratings["aerial_dominance_index_per90"] = pd.to_numeric(df_ratings["aerial_dominance_index"], errors="coerce")




C:\Users\Germán Llorente\AppData\Local\Temp\ipykernel_37532\900931858.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jugadores_total["aerial_dominance_index_per90"] = pd.to_numeric(jugadores_total["aerial_dominance_index"], errors="coerce")


In [167]:
jugadores_total

,player_id,player_name,team,team_id,posiciones,partidos_jugados,overall,potential,value_eur,wage_eur,...,chances_created_per90,goals_conceded_per90,xg_total_per90,goals_minus_xg_per90,PSxG_per90,psxg_minus_goals_conceded_per90,yellow_cards_per90,red_cards_per90,main_position,aerial_dominance_index_per90
0,3023,Yuri Berchiche Izeta,Real Sociedad,210,['Left Back'],21,71,76,1800000.00,20000.00,...,0.78,0.00,0.01,-0.02,0.00,0.00,0.46,0.00,Left Back,0.0
1,3063,Danilo Luiz da Silva,Real Madrid,220,"['Right Back', 'Left Back', 'Right Wing']",24,80,86,16500000.00,90000.00,...,0.71,0.00,0.04,0.04,0.00,0.00,0.25,0.00,Right Back,0.0
2,3084,Christian Atsu Twasam,Málaga,223,"['Left Midfield', 'Right Midfield', 'Left Cent...",11,63,69,375000.00,4000.00,...,1.95,0.00,0.41,0.24,0.00,0.00,0.00,0.00,Right Midfield,0.0
3,3130,Gaël Kakuta,Sevilla,213,"['Left Midfield', 'Right Wing']",2,76,78,6500000.00,60000.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.96,0.00,Left Midfield,0.0
4,3160,Rene Krhin,Granada,1049,"['Right Defensive Midfield', 'Left Defensive M...",23,75,80,4500000.00,35000.00,...,0.24,0.00,0.03,-0.04,0.00,0.00,0.18,0.00,Right Defensive Midfield,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1903,7788,Ciro Immobile,Torino,241,"['Left Center Forward', 'Right Center Forward'...",14,78,85,12000000.00,50000.00,...,1.84,0.00,0.50,-0.07,0.00,0.00,0.17,0.00,Left Center Forward,0.0
1904,7852,Tim Matavž,Genoa,233,"['Right Defensive Midfield', 'Right Wing', 'Ce...",7,81,88,20500000.00,70000.00,...,0.77,0.00,0.14,-0.15,0.00,0.00,0.00,0.00,Left Center Forward,0.0
1905,8217,Kingsley Coman,Juventus,224,['Left Center Forward'],1,75,75,2600000.00,50000.00,...,1.87,0.00,0.15,-0.15,0.00,0.00,0.00,0.00,Left Center Forward,0.0
1906,27853,Mauro Matías Zárate,Fiorentina,239,"['Center Forward', 'Right Center Forward', 'Le...",15,79,80,11000000.00,90000.00,...,1.61,0.00,0.43,0.04,0.00,0.00,0.32,0.16,Left Wing,0.0


In [168]:
jugadores_total["main_position"].unique()

array(['Left Back', 'Right Back', 'Right Midfield', 'Left Midfield',
       'Right Defensive Midfield', 'Left Defensive Midfield',
       'Goalkeeper', 'Right Center Back', 'Center Forward',
       'Center Attacking Midfield', 'Right Wing', 'Left Wing',
       'Left Center Back', 'Left Center Midfield',
       'Center Defensive Midfield', 'Right Center Midfield',
       'Right Center Forward', 'Left Center Forward', 'Right Wing Back',
       'Right Attacking Midfield', 'Center Back', 'Left Wing Back'],
      dtype=object)

In [169]:
# Variables que quieres normalizar
metricas_minmax = [
    'tackle_success_rate', 'interception_success_rate', 'head_clearance_success_rate',
    'headed_shot_duel_rate', 'pass_completion_rate', 'avg_pass_length', 'progressive_passes_accuracy',
    'passes_final_third_accuracy', 'passes_to_box_accuracy', 'progressive_carries_rate',
    'through_balls_completed', 'avg_carry_distance', 'duel_success_rate', 'dribble_success_rate',
    'minutes_played', 'shot_accuracy', 'avg_xg', 'conversion_rate_inside_box', 'save_percentage',
    'tackles_successful_per90', 'interceptions_per90', 'clearances_per90', 'blocks_per90',
    'head_clearances_per90', 'head_clearances_won_per90', 'times_dribbled_past_per90',
    'ball_recoveries_per90', 'offensive_recoveries_per90', 'pressures_per90', 'counterpress_per90',
    'total_passes_per90', 'completed_passes_per90', 'incomplete_passes_per90', 'passes_out_per90',
    'offside_passes_per90', 'failed_passes_per90', 'ground_passes_per90', 'low_passes_per90',
    'high_passes_per90', 'deflected_passes_per90', 'crosses_total_per90', 'crosses_completed_per90',
    'cutbacks_total_per90', 'cutbacks_completed_per90', 'switches_total_per90', 'switches_completed_per90',
    'through_balls_total_per90', 'through_balls_completed_per90', 'passes_own_half_per90',
    'passes_opposition_half_per90', 'progressive_passes_per90', 'progressive_passes_completed_per90',
    'passes_final_third_per90', 'passes_to_box_per90', 'carries_per90', 'progressive_carries_per90',
    'dribbles_completed_per90', 'duels_total_per90', 'duels_won_per90', 'fouls_won_per90',
    'fouls_committed_per90', 'penalties_won_per90', 'penalties_conceded_per90', 'penalties_saved_per90',
    'saves_per90', 'keeper_sweeper_per90', 'shots_total_per90', 'non_blocked_shots_per90',
    'shots_on_target_per90', 'shots_off_target_per90', 'blocked_shots_per90', 'shots_inside_box_per90',
    'free_kick_goals_per90', 'goals_scored_per90', 'penalty_goals_per90', 'goal_assists_per90',
    'key_passes_per90', 'chances_created_per90', 'goals_conceded_per90', 'xg_total_per90',
    'goals_minus_xg_per90', 'PSxG_per90', 'psxg_minus_goals_conceded_per90', 'yellow_cards_per90',
    'red_cards_per90', 'penalty_save_percentage', 'aerial_dominance_index_per90'
]


In [170]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler


# Diccionario de variables por posición
estadisticas_posiciones = {
    "goalkeeper": ["saves_per90", "save_percentage", "psxg_minus_goals_conceded_per90", 
                   "keeper_sweeper_per90", "penalty_save_percentage", "aerial_dominance_index_per90", "progressive_pases_per90", "pass_completion_rate"],
    "center_back": ["tackle_success_rate", "tackles_successful_per90", "interceptions_per90", "interception_success_rate", "clearances_per90", 
                    "blocks_per90", "duels_won_per90", "times_dribbled_past_per90", "progressive_passes_per90", "passes_completed_per90", "duel_success_rate", "goals_scored_per90", "chances_created_per90"],
    "side_back": ["tackle_success_rate", "tackles_successful_per90", "interceptions_per90", "interception_success_rate", "pressures_per90", "chances_created_per90",
                  "progressive_carries_per90", "crosses_completed_per90", "duels_won_per90", "fouls_committed_per90", "goals_scored_per90", "goal_assists_per90"],
    "defensive_midfield": ["interceptions_per90", "ball_recoveries_per90", "pressures_per90", "chances_created_per90", 
                           "progressive_passes_per90", "duels_won_per90", "fouls_committed_per90", "goals_scored_per90", "goal_assists_per90", "duel_success_rate"],
    "center_midfield": ["progressive_passes_per90", "chances_created_per90", "dribbles_completed_per90", 
                        "duels_won_per90", "pressures_per90", "goals_scored_per90", "goal_assists_per90", "dribble_success_rate", "duel_success_rate", "ball_recoveries_per90"],
    "offensive_midfield": ["goal_assists_per90", "key_passes_per90", "progressive_passes_per90", "duels_won_per90", "duel_success_rate",
                           "shots_total_per90", "xg_total_per90", "dribbles_completed_per90", "goals_scored_per90", "dribble_success_rate", "progressive_carries_per90"],
    "winger": ["dribbles_completed_per90", "dribble_success_rate", "crosses_completed_per90", "key_passes_per90", "ball_recoveries_per90", "pressures_per90", "fouls_won_per90",
               "goal_assists_per90", "progressive_carries_per90", "goals_scored_per90", "goals_minus_xg_per90", "progressive_carries_rate"],
    "striker": ["xg_total_per90", "goals_minus_xg_per90", "goals_scored_per90", "shot_accuracy", "chances_created_per90", "dribble_success_rate", "dribbles_completed_per90",
                "goal_assists_per90", "penalties_won_per90", "key_passes_per90", "progressive_carries_rate", "headed_shot_duel_rate", "fouls_won_per90", "progressive_carries_per90", "pass_completion_rate", "ball_recoveries_per90", "pressures_per90"]
}

# Diccionario de etiquetas por posición
etiquetas_posiciones = {
    "goalkeeper": ["Goalkeeper"],
    "center_back": ['Center Back', 'Right Center Back', 'Left Center Back'],
    "side_back": ['Left Back', 'Right Back', 'Right Wing Back', 'Left Wing Back'],
    "defensive_midfield": ['Right Defensive Midfield', 'Center Defensive Midfield', 'Left Defensive Midfield'],
    "center_midfield": ['Right Center Midfield', 'Center Midfield', 'Left Center Midfield'],
    "offensive_midfield": ['Left Attacking Midfield', 'Center Attacking Midfield', 'Right Attacking Midfield'],
    "winger": ['Left Wing', 'Left Midfield', 'Right Wing', 'Right Midfield'],
    "striker": ['Left Center Forward', 'Right Center Forward','Center Forward', 'Secondary Striker']
}

# Variables que quieres conservar tal cual
variables_extra = ['main_position', 'player_id', 'player_name', 'team', 'team_id', 'minutes_played']


In [171]:
import pandas as pd

def normalizar_metricas_percentiles(
    df, 
    metricas, 
    variables_extra, 
    posiciones_filtrar=None,
    min_minutos=850 #25% de una temporada
):
    """
    Normaliza métricas usando percentiles (0-100).
    
    Parámetros:
    - df: DataFrame original
    - metricas: lista de columnas a normalizar
    - variables_extra: lista de columnas a mantener sin cambios
    - posiciones_filtrar: None (no filtra) o lista de valores de 'main_position' a filtrar
    
    Devuelve:
    - DataFrame normalizado por percentiles
    """
        # Filtrar por minutos jugados
    if "minutes_played" in df.columns:
        df = df[df['minutes_played'] >= min_minutos]
    
    # Si se especifican posiciones, filtramos
    if posiciones_filtrar is not None:
        df = df[df['main_position'].isin(posiciones_filtrar)].copy()
    
    # Comprobamos qué métricas existen realmente en el DataFrame
    columnas_existentes = [col for col in metricas if col in df.columns]
    
    # Crear nuevo DataFrame con métricas y variables extra
    df_percentiles = df[variables_extra + columnas_existentes].copy()
    
    # Convertimos a numérico las columnas de métricas
    for col in columnas_existentes:
        df_percentiles[col] = pd.to_numeric(df_percentiles[col], errors='coerce')
    
    # Aplicar percentiles (0-100)
    for col in columnas_existentes:
        df_percentiles[col] = df_percentiles[col].rank(pct=True) * 100

    return df_percentiles

df_porteros_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["goalkeeper"], variables_extra, etiquetas_posiciones["goalkeeper"])
df_laterales_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["side_back"], variables_extra, etiquetas_posiciones["side_back"])
df_centrales_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["center_back"], variables_extra, etiquetas_posiciones["center_back"])
df_centrocampistas_defensivo_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["defensive_midfield"], variables_extra, etiquetas_posiciones["defensive_midfield"])
df_centrocampistas_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["center_midfield"], variables_extra, etiquetas_posiciones["center_midfield"])
df_centrocampistas_ofensivo_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["offensive_midfield"], variables_extra, etiquetas_posiciones["offensive_midfield"])
df_extremos_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["winger"], variables_extra, etiquetas_posiciones["winger"])
df_delanteros_percentiles = normalizar_metricas_percentiles(jugadores_total, estadisticas_posiciones["striker"], variables_extra, etiquetas_posiciones["striker"])


In [172]:
df_delanteros_percentiles[df_delanteros_percentiles["main_position"] == "Left Center Forward"]

,main_position,player_id,player_name,team,team_id,minutes_played,xg_total_per90,goals_minus_xg_per90,goals_scored_per90,shot_accuracy,...,goal_assists_per90,penalties_won_per90,key_passes_per90,progressive_carries_rate,headed_shot_duel_rate,fouls_won_per90,progressive_carries_per90,pass_completion_rate,ball_recoveries_per90,pressures_per90
85,Left Center Forward,6382,Fernando José Torres Sanz,Atlético Madrid,212,1582,72.333333,92.000000,89.333333,49.000000,...,54.000000,26.666667,43.333333,92.666667,86.000000,79.666667,91.333333,25.333333,70.666667,62.333333
166,Left Center Forward,6707,Charles Días Barbosa de Oliveira,Málaga,223,3067,37.666667,60.333333,47.666667,69.333333,...,37.333333,26.666667,59.333333,76.000000,58.000000,37.000000,31.333333,50.000000,25.000000,26.000000
181,Left Center Forward,6735,Adrián López Álvarez,Villarreal,222,930,23.666667,84.333333,54.666667,99.333333,...,98.666667,26.666667,83.000000,83.333333,2.333333,16.333333,78.000000,30.666667,9.000000,23.333333
415,Left Center Forward,26009,Roberto Soldado Rillo,Villarreal,222,2518,7.000000,45.000000,11.666667,71.333333,...,97.333333,62.666667,24.000000,64.000000,31.000000,83.333333,42.333333,33.333333,33.666667,30.000000
758,Left Center Forward,4630,Saido Berahino,West Bromwich Albion,27,1598,47.000000,12.666667,18.666667,21.000000,...,6.666667,76.666667,70.333333,18.000000,5.000000,45.333333,70.000000,82.000000,89.333333,43.333333
783,Left Center Forward,5458,Odion Jude Ighalo,Watford,23,3390,72.333333,27.000000,58.333333,78.333333,...,20.333333,76.666667,48.000000,70.000000,23.333333,46.000000,61.333333,79.333333,58.666667,32.666667
856,Left Center Forward,10960,Jamie Vardy,Leicester City,22,3394,93.333333,64.333333,90.666667,66.000000,...,61.666667,99.666667,64.333333,89.333333,67.666667,8.666667,62.666667,9.333333,31.000000,31.333333
978,Left Center Forward,3434,Stevan Jovetić,Inter Milan,238,1685,63.666667,17.666667,40.666667,63.333333,...,48.333333,26.666667,95.666667,58.666667,45.333333,91.666667,98.000000,55.000000,94.000000,38.333333
979,Left Center Forward,3477,Álvaro Borja Morata Martín,Juventus,224,1495,63.666667,45.000000,65.333333,71.333333,...,98.000000,96.666667,80.000000,59.333333,45.333333,97.333333,92.000000,80.000000,73.333333,86.000000
1000,Left Center Forward,5472,Mario Mandžukić,Juventus,224,2234,39.666667,74.000000,61.666667,42.666667,...,72.666667,90.000000,61.666667,72.666667,96.666667,35.000000,13.333333,48.666667,4.666667,77.333333


In [173]:
def normalizar_metricas_minmax(df, metricas, variables_extra, posiciones_filtrar=None, min_minutos= 850):
    """
    Normaliza métricas usando min-max scaling (0-100).

    Parámetros:
    - df: DataFrame original
    - metricas: lista de columnas a normalizar
    - variables_extra: lista de columnas a mantener sin cambios
    - posiciones_filtrar: None o lista de posiciones de 'main_position' a filtrar

    Devuelve:
    - DataFrame normalizado con valores entre 0 y 100
    """
        # Filtrar por minutos jugados
    if "minutes_played" in df.columns:
        df = df[df['minutes_played'] >= min_minutos]

    # Filtrado opcional
    if posiciones_filtrar is not None:
        df = df[df['main_position'].isin(posiciones_filtrar)].copy()
    
    # Seleccionar columnas existentes
    columnas_existentes = [col for col in metricas if col in df.columns]
    
    # Crear DataFrame nuevo
    df_minmax = df[variables_extra + columnas_existentes].copy()
    
    # Convertir métricas a numérico
    for col in columnas_existentes:
        df_minmax[col] = pd.to_numeric(df_minmax[col], errors='coerce')
    
    # Aplicar normalización min-max
    for col in columnas_existentes:
        col_values = df_minmax[col]
        min_val = col_values.min()
        max_val = col_values.max()
        if pd.notnull(min_val) and pd.notnull(max_val) and max_val != min_val:
            df_minmax[col] = (col_values - min_val) / (max_val - min_val) * 100
        else:
            df_minmax[col] = 0  # Para columnas constantes o vacías

    return df_minmax

df_porteros_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["goalkeeper"], variables_extra, etiquetas_posiciones["goalkeeper"])
df_laterales_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["side_back"], variables_extra, etiquetas_posiciones["side_back"])
df_centrales_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["center_back"], variables_extra, etiquetas_posiciones["center_back"])
df_centrocampistas_defensivo_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["defensive_midfield"], variables_extra, etiquetas_posiciones["defensive_midfield"])
df_centrocampistas_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["center_midfield"], variables_extra, etiquetas_posiciones["center_midfield"])
df_centrocampistas_ofensivo_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["offensive_midfield"], variables_extra, etiquetas_posiciones["offensive_midfield"])
df_extremos_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["winger"], variables_extra, etiquetas_posiciones["winger"])
df_delanteros_minmax = normalizar_metricas_minmax(jugadores_total, estadisticas_posiciones["striker"], variables_extra, etiquetas_posiciones["striker"])

In [174]:
def calcular_rating(df):
    """
    Calcula el rating ponderado de los jugadores según su posición principal ('main_position'),
    usando las ponderaciones definidas por el usuario.

    Parámetros:
    - df: DataFrame normalizado (min-max o percentiles)
    - tipo: 'minmax' o 'percentiles' (solo para referencia, no afecta al cálculo ahora)

    Devuelve:
    - DataFrame con una nueva columna 'rating'
    """

    # Ponderaciones actualizadas por "grupo" de posiciones (las tuyas)
    ponderaciones = {
        'goalkeeper': {
            'saves_per90': 0.075,
            'save_percentage': 0.25,
            'psxg_minus_goals_conceded_per90': 0.425,
            'keeper_sweeper_per90': 0.075,
            'penalty_save_percentage': 0.05,
            'aerial_dominance_index_per90': 0.075,
            'progressive_passes_per90': 0.05,
            'pass_completion_rate': 0.05
        },
        'center_back': {
            'interceptions_per90': 0.15,
            'clearances_per90': 0.15,
            'blocks_per90': 0.10,
            'tackle_success_rate': 0.05,
            'duels_won_per90': 0.10,
            'duel_success_rate': 0.10,
            'interception_success_rate': 0.05,
            'times_dribbled_past_per90': 0.10,  # ¡Invertir!
            'passes_completed_per90': 0.025,
            'progressive_passes_per90': 0.075,
            'tackles_successful_per90': 0.05,
            'goals_scored_per90': 0.05
        },
        'side_back': {
            'pressures_per90': 0.15,
            'progressive_carries_per90': 0.15,
            'crosses_completed_per90': 0.15,
            'duels_won_per90': 0.10,
            'tackle_success_rate': 0.10,
            'interception_success_rate': 0.05,
            'tackles_successful_per90': 0.05,
            'fouls_committed_per90': 0.05,  # ¡Invertir!
            'goal_assists_per90': 0.05,
            'chances_created_per90': 0.05,
            'goals_scored_per90': 0.05
        },
        'defensive_midfield': {
            'ball_recoveries_per90': 0.20,
            'interceptions_per90': 0.15,
            'pressures_per90': 0.15,
            'duels_won_per90': 0.10,
            'duel_success_rate': 0.10,
            'progressive_passes_per90': 0.10,
            'fouls_committed_per90': 0.05,  # ¡Invertir!
            'chances_created_per90': 0.10,
            'goals_scored_per90': 0.05
        },
        'center_midfield': {
            'progressive_passes_per90': 0.20,
            'chances_created_per90': 0.20,
            'dribbles_completed_per90': 0.05,
            'dribble_success_rate': 0.10,
            'duels_won_per90': 0.10,
            'duel_success_rate': 0.10,
            'pressures_per90': 0.10,
            'goal_assists_per90': 0.05,
            'goals_scored_per90': 0.05
        },
        'offensive_midfield': {
            'goal_assists_per90': 0.20,
            'key_passes_per90': 0.20,
            'chances_created_per90': 0.20,
            'shots_total_per90': 0.05,
            'xg_total_per90': 0.10,
            'dribbles_completed_per90': 0.10,
            'dribble_success_rate': 0.05,
            'goals_scored_per90': 0.10
        },
        'winger': {
            'dribbles_completed_per90': 0.10,
            'dribble_success_rate': 0.10,
            'progressive_carries_per90': 0.025,
            'crosses_completed_per90': 0.5,
            'chances_created_per90': 0.10,
            'goal_assists_per90': 0.10,
            'goals_scored_per90': 0.20,
            'goals_minus_xg_per90': 0.20,
            'progressive_carries_rate': 0.05,
            'ball_recoveries_per90': 0.025,
            'pressures_per90': 0.025,
            'fouls_won_per90': 0.025
        },
        'striker': {
            'xg_total_per90': 0.05,
            'goals_scored_per90': 0.20,
            'shot_accuracy': 0.05,
            'goals_minus_xg_per90': 0.20,
            'headed_shot_duel_rate': 0.05,
            'dribbles_completed': 0.05,
            'chances_created_per90': 0.075,
            'goal_assists_per90': 0.05,
            'fouls_won_per90': 0.05,
            'progressive_carries_per90': 0.075,
            'pass_completion_rate': 0.075,
            'ball_recoveries_per90': 0.05,
            'pressures_per90': 0.025
        }
    }

    # Definir mapeo de etiquetas
    etiquetas_posiciones = {
        "goalkeeper": ["Goalkeeper"],
        "center_back": ['Center Back', 'Right Center Back', 'Left Center Back'],
        "side_back": ['Left Back', 'Right Back', 'Right Wing Back', 'Left Wing Back'],
        "defensive_midfield": ['Right Defensive Midfield', 'Center Defensive Midfield', 'Left Defensive Midfield'],
        "center_midfield": ['Right Center Midfield', 'Center Midfield', 'Left Center Midfield'],
        "offensive_midfield": ['Left Attacking Midfield', 'Center Attacking Midfield', 'Right Attacking Midfield'],
        "winger": ['Left Wing', 'Left Midfield', 'Right Wing', 'Right Midfield'],
        "striker": ['Center Forward', 'Secondary Striker', 'Left Center Forward', 'Right Center Forward']
    }

    # Invertir el diccionario para encontrar la categoría desde la etiqueta
    etiqueta_a_categoria = {}
    for categoria, etiquetas in etiquetas_posiciones.items():
        for etiqueta in etiquetas:
            etiqueta_a_categoria[etiqueta] = categoria

    # Calcular ratings
    df = df.copy()
    ratings = []

    for idx, row in df.iterrows():
        main_position = row['main_position']
        rating = 0

        if main_position in etiqueta_a_categoria:
            categoria = etiqueta_a_categoria[main_position]
            if categoria in ponderaciones:
                for stat, peso in ponderaciones[categoria].items():
                    if stat in row:
                        valor = row[stat]
                        # Invertimos métricas "negativas"
                        if stat in ['times_dribbled_past_per90', 'fouls_committed_per90']:
                            valor = 100 - valor
                        rating += valor * peso
        ratings.append(rating)

    df['rating'] = ratings
    return df


In [175]:
df_porteros_minmax = calcular_rating(df_porteros_minmax)
df_laterales_minmax = calcular_rating(df_laterales_minmax)
df_centrales_minmax = calcular_rating(df_centrales_minmax)
df_centrocampistas_defensivo_minmax = calcular_rating(df_centrocampistas_defensivo_minmax)
df_centrocampistas_minmax = calcular_rating(df_centrocampistas_minmax)
df_centrocampistas_ofensivo_minmax = calcular_rating(df_centrocampistas_ofensivo_minmax)
df_extremos_minmax = calcular_rating(df_extremos_minmax)
df_delanteros_minmax = calcular_rating(df_delanteros_minmax)


In [176]:
df_porteros_minmax

,main_position,player_id,player_name,team,team_id,minutes_played,saves_per90,save_percentage,psxg_minus_goals_conceded_per90,keeper_sweeper_per90,penalty_save_percentage,aerial_dominance_index_per90,pass_completion_rate,rating
10,Goalkeeper,3379,Alphonse Areola,Villarreal,222,2984,39.249147,77.826962,85.106383,43.005181,50.00,56.906077,39.488762,70.538422
26,Goalkeeper,4691,Claudio Andrés Bravo Muñoz,Barcelona,217,2957,39.931741,92.877264,96.808511,13.471503,0.00,6.077348,100.000000,73.823977
58,Goalkeeper,5577,Francisco Guillermo Ochoa Magaña,Málaga,223,928,49.829352,67.847082,96.808511,29.533679,0.00,4.005525,40.788894,66.397474
61,Goalkeeper,5597,Keylor Navas Gamboa,Real Madrid,220,3152,45.392491,78.672032,92.553191,49.222798,75.00,40.469613,67.452622,76.257113
82,Goalkeeper,6378,Jan Oblak,Atlético Madrid,212,3535,26.621160,100.000000,96.808511,20.725389,0.00,31.215470,29.660643,73.518801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1687,Goalkeeper,8947,Ron-Robert Zieler,Hannover 96,178,3138,93.856655,50.663984,40.425532,64.248705,0.00,60.082873,67.254297,49.573679
1693,Goalkeeper,9082,Przemysław Tytoń,VfB Stuttgart,174,2770,81.228669,40.845070,30.851064,30.051813,25.00,20.580110,47.289555,36.826992
1760,Goalkeeper,12319,Ramazan Özcan,Ingolstadt,189,2593,23.549488,38.430584,63.829787,73.575130,0.00,50.276243,19.413839,48.761062
1779,Goalkeeper,15581,Bernd Leno,Bayer Leverkusen,904,3068,50.853242,59.919517,71.276596,65.803109,16.67,52.624309,62.230057,61.913485


In [ ]:
# Primero importamos os
import os

# Definir la ruta donde guardar
ruta_guardado = "datos/normalizaciones_posicion/minmax/"

# Crear la carpeta si no existe
os.makedirs(ruta_guardado, exist_ok=True)

# Guardar cada DataFrame como CSV
df_porteros_minmax.to_csv(os.path.join(ruta_guardado, "porteros_minmax.csv"), index=False)
df_laterales_minmax.to_csv(os.path.join(ruta_guardado, "laterales_minmax.csv"), index=False)
df_centrales_minmax.to_csv(os.path.join(ruta_guardado, "centrales_minmax.csv"), index=False)
df_centrocampistas_defensivo_minmax.to_csv(os.path.join(ruta_guardado, "centrocampistas_defensivos_minmax.csv"), index=False)
df_centrocampistas_minmax.to_csv(os.path.join(ruta_guardado, "centrocampistas_minmax.csv"), index=False)
df_centrocampistas_ofensivo_minmax.to_csv(os.path.join(ruta_guardado, "centrocampistas_ofensivos_minmax.csv"), index=False)
df_extremos_minmax.to_csv(os.path.join(ruta_guardado, "extremos_minmax.csv"), index=False)
df_delanteros_minmax.to_csv(os.path.join(ruta_guardado, "delanteros_minmax.csv"), index=False)